In [1]:
import findspark
findspark.init('/home/spark/spark-2.1.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("CorretingLenRegProj").getOrCreate()

In [6]:
df = spark.read.csv("cruise_ship_info.csv", inferSchema = True , header = True)

In [11]:
for i in df.head(7):
    print(i)
    print("\n")
df.head(5)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)


Row(Ship_name='Ecstasy', Cruise_line='Carnival', Age=22, Tonnage=70.367, passengers=20.52, length=8.55, cabins=10.2, passenger_density=34.29, crew=9.2)


Row(Ship_name='Elation', Cruise_line='Carnival', A

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)]

In [20]:
df.groupby("Cruise_line" ).count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [21]:
from pyspark.ml.feature import StringIndexer

In [23]:
indexer = StringIndexer(inputCol = 'Cruise_line', outputCol = 'curise_cat')
indexed = indexer.fit(df).transform(df)
indexed.head(7)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, curise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, curise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, curise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, curise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, curise_cat=1.0),
 Row(Ship_name='Ecstasy', Cruise_line='Carnival', Age=22, Tonnage=70.367, passengers=20.52, length=8.55, cabins=10.2, pass

In [25]:
indexed.groupby("curise_cat" ).count().show()

+----------+-----+
|curise_cat|count|
+----------+-----+
|       8.0|    6|
|       0.0|   23|
|       7.0|    8|
|      18.0|    2|
|       1.0|   22|
|       4.0|   13|
|      11.0|    4|
|      14.0|    3|
|       3.0|   14|
|      19.0|    1|
|       2.0|   17|
|      17.0|    2|
|      10.0|    5|
|      13.0|    3|
|       6.0|   10|
|       5.0|   11|
|      15.0|    3|
|       9.0|    6|
|      16.0|    2|
|      12.0|    3|
+----------+-----+



In [26]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [27]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'curise_cat']

In [29]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'curise_cat'],outputCol = 'features')

In [30]:
output = assembler.transform(indexed)

In [38]:
#output.show()
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [39]:
final_data = output.select('features','crew')

In [40]:
final_data.head(5)

[Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), crew=3.55),
 Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), crew=3.55),
 Row(features=DenseVector([26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0]), crew=6.7),
 Row(features=DenseVector([11.0, 110.0, 29.74, 9.53, 14.88, 36.99, 1.0]), crew=19.1),
 Row(features=DenseVector([17.0, 101.353, 26.42, 8.92, 13.21, 38.36, 1.0]), crew=10.0)]

In [42]:
train_data , test_data  =  final_data.randomSplit([0.7,0.3])

In [44]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               116|
|   mean| 7.979827586206908|
| stddev|3.5854808622405514|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [46]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               42|
|   mean|7.281428571428571|
| stddev|3.252169685841319|
|    min|             0.88|
|    max|             13.6|
+-------+-----------------+



In [47]:
from pyspark.ml.regression import LinearRegression 

In [49]:
ship_lr = LinearRegression(labelCol = 'crew')

In [50]:
trained_ship_model = ship_lr.fit(train_data)

In [65]:
ship_results = trained_ship_model.evaluate(test_data)


In [67]:
ship_results.rootMeanSquaredError
print('\n')
ship_results.r2

0.9450762494663641

In [53]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               116|
|   mean| 7.979827586206908|
| stddev|3.5854808622405514|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [68]:
ship_results.meanAbsoluteError

0.5957421878128166

In [69]:
ship_results.meanSquaredError

0.5670758427953894

In [71]:
from pyspark.sql.functions import corr

In [73]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

